# Summary

<h3>Data & Methodology</h3>

In each section we should have:
1. Brief description
2. Challenges & limitations
3. Preprocessing
4. Print final dataset at the end

**Web scraping**


Web scraping from Domain.com

**Open Route Service Data**

ORS and merging scraped data with the ORS data

**Census Data**

Census data & converting SA2 to postcode

**School Data**

Brief description

**Summary**

List of final datasets & all features?

<h3>Questions & Analysis</h3>

Sections should basically be a summary of each individual notebook including overall method, final results and graphs

**Question 1 - What are the most important internal and external features in predicting rental prices?**

Graphs would be great here, and the notes at the end of analysis_preoprties.ipynb can be copied in

**Question 2 - What are the top 10 suburbs with the highest predicted growth rate?**

Not sure how we should do this. Code could be turned into a function and loaded here to generate our growth rates.

Forecasting graph should be here as well

**Question 3 - What are the most liveable and affordable suburbs according to your chosen metrics?**

We could probably just have a high level explanation here including formulas, and then saving the liveability & affordability results to load in here

<h3>Assumptions & Business Context</h3>

Assumptions can either be in this section or scattered throughout questions & analysis

Would be good to be able to use tool from this notebook if tooluse.py could be converted into a function